# Importe das bibliotecas necessárias

In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import data as wb
import matplotlib.pyplot as plt

## Risco individual do Ativo BBDC3 - Bradesco

In [ ]:
bbdc_prices = wb.DataReader('BBDC3.SA', data_source = 'yahoo', start='2019-01-01')
bbdc_prices.head()

### Interessa apenas a última coluna do dataset que é [Adj Close], pois nos oferece dados ajustados conforme o pagamento de dividendos. Iremos eliminar as demais colunas (High, Low, Open, Close e Volume), e renomear para BBDC:

In [ ]:
# Renomeia a coluna adj close
bbdc_prices.rename(columns = {"Adj Close":"BBDC"}, inplace = True)

# Remove as colunas
bbdc = bbdc_prices.drop(bbdc_prices.columns[[0,1,2,3,4]], axis = 1)

## Retornos da série histórica de preços da BBDC3 - (Preço Final – Preço inicial)/Preço Final - variações percentuais:

In [ ]:
bbdc_returns = bbdc.pct_change()

In [ ]:
bbdc.returns.head()

## “log-return” -> retornos logarítmicos dos preços. Representam melhor os retornos acumulados que o retorno linear:

In [ ]:
# Retornos de LOG diários
bbdc_log_returns = np.log(bbdc/bbdc.shift(1))

## Gráfico dos retornos para observar o comportamento:

In [ ]:
# Plot do gráfico de retornos
bbdc_log_returns.plot()

## Plot dos retornos em histograma:

In [ ]:
# Gráfico de distribuição dos retornos
bbdc_log_returns.plot.hist(bins = 60)
plt.show()

## Calculando o risco individual de cada ativo através do desvio padrão:

In [ ]:
# Volatilidade diária
volatilidade_diaria = np.std(bbdc_returns['BBDC'])

## “Printando” a volatilidade diária:

In [ ]:
print(volatilidade_diaria)

## Volatilidade anual:

In [ ]:
volatilidade_anual = volatilidade_diaria*np.sqrt(252)
print(volatilidade_anual)

## Calcular o desvio padrão dos retornos que estão abaixo de zero. 
## Ou seja, aqueles retornos que representam as perdas diárias:

In [ ]:
semivariancia = np.std(bbdc_returns['BBDC'] < 0)
print(semivariancia)

## Calcular os DRAWDOWN -> máxima queda que os retornos acumulados tiveram em um determinado período, podendo ser analisado como a queda máxima da semana, do mês, do ano ou de um período específico.

### 1 - Calcular os retornos acumulados em base 100:

In [ ]:
retorno_acumulado = bbdc_log_returns*100

### 2 - Retirar os NaNs e substituí-los por zero:

In [ ]:
retorno_acumulado = retorno_acumulado['BBDC'].fillna(0)

### 3 - Calcular o ponto máximo dos acumulados utilizando o Numpy e garantir que não fique abaixo de 1:

In [ ]:
ponto_maximo = np.maximum.accumulate(retorno_acumulado)
ponto_maximo[ponto_maximo < 1] = 1

### 4 - Calcular o drawdown:

In [ ]:
drawdown = (retorno_acumulado)/running_max - 1

### 5 - Plotar para ver o resultado:

In [ ]:
drawdown.plot()
<matplotlib.axes._subplots.AxesSubplot at 0x1236d5f8>

# CAPM e Beta

## Beta

## Para calcular o beta iremos utilizar os seguintes passos:

### 1 - Importar o Ibovespa e dar o mesmo tratamento de dados anterior que demos a BBDC3:

In [ ]:
ibov = wb.DataReader('^IBVSP', data_source = 'yahoo', start = '2019-01-01')
ibov.rename(columns = {"Adj Close":"IBOV"}, inplace = True)
ibov = ibov.drop(ibov.columns[[0,1,2,3,4]], axis = 1)
ibov_returns = np.log(ibov/ibov.shift(1))

### 2 - Calcular a variância do ibov:

In [ ]:
ibov_var = ibov_returns.var()

### 3 - Juntar os dataframes:

In [ ]:
from functools import reduce
join_bbdc_ibov = ([bbdc_log_returns, ibov_returns])
cov_bbdc = reduce(lambda left, right: pd.merge(left, right, on = ['Date'], how = 'inner'), join_bbdc_ibov)

### 4 - Montar a matriz de covariância [BBDC,IBOV] e encontrar o coeficiente

In [ ]:
cov_bbdc = cov_bbdc[['BBDC', 'IBOV']].cov
cov_bbdc_coef = cov_bbdc.iloc[0,1]

### 5 - Calcular o beta dividindo o coeficiente de covariância pela variância do Ibov:

In [ ]:
beta_bbdc = cov_bbdc_coef/ibov_var
print(beta_bbdc)

## Calculando o beta através da regressão linear

### 1 - Importar a biblioteca statsmodels:

In [ ]:
import statsmodels.api as sm

### 2 - Definir a variável beta no método OLS (Ordinary Least Squares):

In [ ]:
beta = sm.OLS(bbdc_retornos, ibov_retornos).fit()

### 3 - Chamar o método summary para observar os resultados:

In [ ]:
beta.summary()

# Value at Risk - VaR

### O value at risk é uma medida que mostra para o investidor qual o potencial de perda daquele ativo ou de sua carteira como um todo. O indicador mede o pior cenário que esse ativo ou carteira pode atingir, dadas as condições normais de mercado e um determinado nível de confiança.

## Abordagens para o VaR: Simulação histórica, Variância-Covariância e Paramétrico.

## VaR Histórico

### Percentil de cada retorno correspondente ao VaR e nível de confiança desejado:

In [ ]:
var_90 = np.percentile(bbdc_retornos, 10)
var_95 = np.percentile(bbdc_retornos, 5)
var_99 = np.percentile(bbdc_retornos, 1)

## VaR Paramétrico

### 1 - Importar a função norm do pacote Scipy:

In [ ]:
from scipy.stats import norm

### 2 - Calcular as médias dos retornos através da variável media:

In [ ]:
media = np.mean(bbdc_retornos['BBDC'])

### 3 - Definir as variáveis dos intervalos de confiança:

In [ ]:
VaR_90 = norm.ppf(conf_90, media, vol_bbdc)
VaR_95 = norm.ppf(conf_95, media, vol_bbdc)
VaR_99 = norm.ppf(conf_99, media, vol_bbdc)

## VaR Simulação de Monte Carlo

### Simulação de Monte Carlo em Python:

### 1) Definir o dataset retornos_simulados em branco para abrigar as iterações da simulação:

In [ ]:
retornos_simulados = []

### 2) Criar a variável dias_trade para 252 dias pois estamos analisando retornos diários:

In [ ]:
dias_trade = 252

### 3) Criar um loop de 1000 iterações para a Simulação de Monte Carlo:

In [ ]:
for i in range (1000):
    sim_retornos = np.random.normal(media, vol_bbdc, dias_trade)

### 4) Anexar as iterações no df retornos_simulados:

In [ ]:
retornos_simulados.apped(sim_retornos)

### 5) Calcular o percentil relacionado ao VaR desejado:

In [ ]:
var_99 = np.percentile(retornos_simulados, 1)

# A otimização de portfólio

## Otimizando portfólio com a biblioteca PyPortfólio

### Seleção de Ações:

### 1) Criar uma lista com os tickers das ações (da mesma maneira que está escrito no Yahoo Finance):

In [ ]:
ativos = ['ITSA4.SA', 'PETR4.SA', 'ABEC3.SA', 'VALE3.SA']

### 2) Criar um dataframe vazio para abrigar os dados que vamos coletar:

In [ ]:
df = pd.DataFrame()

### 3) Fazer um loop para preencher o df com os dados das ações com o campo ‘Adj Close’:

In [ ]:
for t in ativos:
    df[t] = wb.DataReader(t, data_source = 'yahoo', start = '2019-01-01', cnd = '2024-01-01')['Adj Close]

### 4) A partir das ações escolhidas, analisar o comportamento dos preços entre o período Janeiro-2018 e Janeiro-2020 graficamente:

In [ ]:
df.plot()

### 5) calcular os retornos diários:

In [ ]:
retorno = df.pct_change()

## Retornos Esperados

#### Separar o df em dois anos (250 dias de pregão) para avaliar qual melhor método de previsão de retornos será utilizado para a série de dados

In [ ]:
df_fut, df_past = df.iloc[:-250], df.iloc[:-250]

## Retorno Médio Histórico

In [ ]:
future_rets_rh = expected_returns.mean_historical_return(df_fut)
mu_rh = expected.returns.return_model(df_past, method = "mean_historical_return")
mean_abs_erros_rh = []
mean_abs_erros_rh.append(np.sum(np.abs(mu_rh-future_rets))/len(mu_rh))

### Média Móvel Exponencial

In [ ]:
future_rets_ema = expected_returns.ema_historical_return(df_fut)
mu_ema = expected.returns.return_model(df_past, method = "ema_historical_return")
mean_abs_erros_ema = []
mean_abs_erros_ema.append(np.sum(np.abs(mu_ema-future_rets))/len(mu_ema))

## Capital Asset Pricing Model (CAPM)

In [ ]:
future_rets_capm = expected_returns.capm_historical_return(df_fut)
mu_capm = expected.returns.return_model(df_past, method = "capm_historical_return")
mean_abs_erros_capm = []
mean_abs_erros_capm.append(np.sum(np.abs(mu_capm-future_rets))/len(mu_capm))

# Retorno Esperado

In [ ]:
from pypfopt import expected_returns

## Preparar os parâmetros de Selic diária e do Ibovespa como carteira de mercado:

In [ ]:
# Calculara a selic diária
selic_aa = 0.019
selic_diaria = (1 + selic_aa)**(1/252)-1

# Ibobespa para o perido passado
ibov = wb.DataReader('^BVSP', data_source = 'yahoo', start = '2017-01-01', end = '2018-12-31')
ibov = ibov.drop(ibov.columns[[0,1,2,3,4]], axis = 1)
ibov

# Previsão dos retornos através do modelo CAPM
re = expected_returns.capm_return(df, market_prices = ibov, risk_free_rate = selic_diaria)
re

## Matriz de Covariância

In [ ]:
from pypfopt import risk_models

### Utilizar o método disponível de achatamento da matriz de covariância e o redutor de erros de Ledoit-Wolf:

In [ ]:
sample_cov = risk_models.CovarianceShrinkage(df).ledoit_wolf()

## Portfólio de Mínima-Volatilidade

### Redução de volatilidade, ou seja, uma combinação de pesos que indique o menor ponto de volatilidade do portfólio:

### Para isso importar o módulo Efficient Frontier

In [ ]:
from pypfopt import EfficientFrontier

### Construir o objeto mv na EfficientFrontier e aplicar o método de min_volatility() para realizar a otimização:

In [ ]:
mv = EfficientFrontier(re, sample_cov)
mv.min_volatility()

### O método clean_weights ajuda a criar um objeto lista para os pesos do portfólio otimizado:

In [ ]:
pesos_vol = mv.clear_weights()
pesos_vol
# O output dessa função são os pesos que o algoritmo de otimização nos dá para cada ativo.

#### O ativo PETR4.SA está com peso zerado, muitas vezes isso pode acontecer devido a disparidade de volatilidade que ele tem perante os outros ativos da carteira, o algoritmo age até o limite da otimização e como não há restrições de peso mínimo ele irá passar exatamete a melhor solução matemática para o problema de otimização.
#### Uma maneira de contornar esses problemas de alocação é através da função de regularização L2 que está dentro do módulo objective_functions:

In [ ]:
from pypfopt import objective_functions

#### Construir novamente a estrutura de mínima volatilidade dessa vez adicionando como objetivo da otimização a função regularizadora com fator gamma de 10%, dessa maneira ela irá tratar esses pesos zero de maneira diferente conferindo a eles alguma distribuição de percentual:

In [ ]:
mv2 = EfficientFrontier(re, sample_cov)
mv2.add_objective(objective_functions.L2_reg, gamma = 0.1)
mv2.min_volativility()
pesos2 = mv_clear_weights()
pesos2
# O output desse bloco de código será o dicionário de pesos.

# Maximização do índice de Sharpe
## Tem como objetivo otimizar a relação prêmio-pelo-risco-retorno através da clássica fórmula de Índice de Sharpe (Taxa livre de risco – Retorno da Carteira)/Volatilidade da Carteira.

In [ ]:
# Maximização de Sharpe
msharpe = EfficientFrontier(re, sample_cov)
msharpe.max_sharpe(risk_free_rate = selic_aa)
sharpe_pesos = msharpe.clearn_weights()
sharpe_pesos
# O output do objeto sharpe_pesos será um dicionário com os pesos otimizados

### Transformar o dicionário em lista para auxiliar nos cálculos:

In [ ]:
lista_sharpe = sharpe_pessos.values()
lista_sharpe = list(lista_sharpe)
lista_sharpe

# Risco Eficiente

#### O modelo de otimização risco eficiente permite que você estabeleça um objetivo de volatilidade para o seu portfólio dessa maneira permitindo que se calcule a máxima rentabilidade, dado esse objetivo. Nesse caso teremos como objetivo 20% de volatilidade:

In [ ]:
r_eficiente = EfficientFrontier(re, sample_cov)
r_eficiente.efficient_risk(target_volatility = 0.2)
r_eficiente_pesos = r_eficiente.clean_weights()
r_eficiente_pesos

# Retorno Eficiente

In [ ]:
re_eficiente = EfficientFrontier(re, sample_cov)
re_eficiente.efficient_risk(target_volatility = 0.25)
re_eficiente_pesos = re_eficiente.clean_weights()
re_eficiente_pesos
# O output serão os pesos otimizados dado o objetivo de retorno

# Modelos de Otimização com restrições

## Restrições Setoriais

In [ ]:
sector_mapper = {
    "BBDC3.SA" : "FINANCEIRO",
    "PETR4.SA" : "COMMODITIES",
    "ABEV3.SA" : "VAREJO",
    "VALE3.SA" ; "COMMODITIES"
}

sector_lower = {"COMMODITIES": 0.05} #>= 5% DE COMODITES
sector_upper = {"VAREJO": 0.10} #<= 10% EM VAREJO

rest_setor = EfficientFrontier(re, sample_cov)
rest_setor.add_sector_constraints(sector_mapper, sector_lower, sector_upper)
rest_setor.max_sharpe()
rest_setor_pesos = rest_sector.clearn_weights()
rest_setor_pesos

## Restrição de ativo específico

In [ ]:
restricao_acao = EfficientFrontier(re, sample_cov)

### Construir as restrições de otimização. Buscar o ticker da ação através de "tickers.index" para trazer exatamente o papel PETR4.SA:

In [ ]:
petr = restricao_acao.tickers.index("PETR4.SA")
restricao_acao.add_constraint(lambda w: w[petr] <= 0.10)

### Por último, realizar a otimização e extrair os pesos selecionados:

In [ ]:
# Otimização
restricao_acao.max_sharpe()

# Pesos
pesos_acao = restricao_acao.clearn_weights()
pesos_acao